<a href="https://colab.research.google.com/github/usm-cos422-522/courseMaterials/blob/main/Lectures/Chapter8-Lecture.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Chapter 8.3 : Step by step

Wide Format:
Person   Age   Weight   Height  
 -------- ----- -------- -------- 
  Bob       32      168      180  
  Alice     24      150      175  
  Steve     64      144      165

Long Format:
Person   Variable   Value  
 -------- ---------- ------- 
  Bob      Age           32  
  Bob      Weight       168  
  Bob      Height       180  
  Alice    Age           24  
  Alice    Weight       150  
  Alice    Height       175  
  Steve    Age           64  
  Steve    Weight       144  
  Steve    Height       165


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
data_url = "https://raw.githubusercontent.com/wesm/pydata-book/2nd-edition/examples/macrodata.csv"
data = pd.read_csv(data_url)

In [ ]:
data.head()

,year,quarter,realgdp,realcons,realinv,realgovt,realdpi,cpi,m1,tbilrate,unemp,pop,infl,realint
0,1959.0,1.0,2710.349,1707.4,286.898,470.045,1886.9,28.98,139.7,2.82,5.8,177.146,0.00,0.00
1,1959.0,2.0,2778.801,1733.7,310.859,481.301,1919.7,29.15,141.7,3.08,5.1,177.830,2.34,0.74
2,1959.0,3.0,2775.488,1751.8,289.226,491.260,1916.4,29.35,140.5,3.82,5.3,178.657,2.74,1.09
3,1959.0,4.0,2785.204,1753.7,299.356,484.052,1931.3,29.37,140.0,4.33,5.6,179.386,0.27,4.06
4,1960.0,1.0,2847.699,1770.5,331.722,462.199,1955.5,29.54,139.6,3.50,5.2,180.007,2.31,1.19


In [ ]:
# PeriodIndex : Immutable ndarray holding ordinal values indicating regular periods in time.
# need to grab the fields now as we are going to cut them out

periods = pd.PeriodIndex(year=data.year, quarter=data.quarter,name='date')
periods

PeriodIndex(['1959Q1', '1959Q2', '1959Q3', '1959Q4', '1960Q1', '1960Q2',
             '1960Q3', '1960Q4', '1961Q1', '1961Q2',
             ...
             '2007Q2', '2007Q3', '2007Q4', '2008Q1', '2008Q2', '2008Q3',
             '2008Q4', '2009Q1', '2009Q2', '2009Q3'],
            dtype='period[Q-DEC]', name='date', length=203, freq='Q-DEC')

In [ ]:
# Select three columns

columns = pd.Index(['realgdp', 'infl', 'unemp'], name='item')
data = data.reindex(columns=columns)
data


item,realgdp,infl,unemp
0,2710.349,0.00,5.8
1,2778.801,2.34,5.1
2,2775.488,2.74,5.3
3,2785.204,0.27,5.6
4,2847.699,2.31,5.2
...,...,...,...
198,13324.600,-3.16,6.0
199,13141.920,-8.79,6.9
200,12925.410,0.94,8.1
201,12901.504,3.37,9.2


In [ ]:
# period.to_timestamp : Return the Timestamp representation of the Period.
# Uses the target frequency specified at the part of the period specified by how, which is either Start or Finish.

#data.index = periods.to_timestamp('D', 'start')
data.index = periods.to_timestamp('D', 'end')
data

item,realgdp,infl,unemp
date,,,
1959-03-31 23:59:59.999999999,2710.349,0.00,5.8
1959-06-30 23:59:59.999999999,2778.801,2.34,5.1
1959-09-30 23:59:59.999999999,2775.488,2.74,5.3
1959-12-31 23:59:59.999999999,2785.204,0.27,5.6
1960-03-31 23:59:59.999999999,2847.699,2.31,5.2
...,...,...,...
2008-09-30 23:59:59.999999999,13324.600,-3.16,6.0
2008-12-31 23:59:59.999999999,13141.920,-8.79,6.9
2009-03-31 23:59:59.999999999,12925.410,0.94,8.1


In [ ]:
pd.melt?

In [ ]:
# Ooops, cannot find the date field (in the index)
melted = pd.melt(data,id_vars='date',var_name='attribute',value_name='value')
melted

KeyError: 'date'

In [ ]:
data.reset_index()

item,date,realgdp,infl,unemp
0,1959-03-31 23:59:59.999999999,2710.349,0.00,5.8
1,1959-06-30 23:59:59.999999999,2778.801,2.34,5.1
2,1959-09-30 23:59:59.999999999,2775.488,2.74,5.3
3,1959-12-31 23:59:59.999999999,2785.204,0.27,5.6
4,1960-03-31 23:59:59.999999999,2847.699,2.31,5.2
...,...,...,...,...
198,2008-09-30 23:59:59.999999999,13324.600,-3.16,6.0
199,2008-12-31 23:59:59.999999999,13141.920,-8.79,6.9
200,2009-03-31 23:59:59.999999999,12925.410,0.94,8.1
201,2009-06-30 23:59:59.999999999,12901.504,3.37,9.2


In [ ]:
melted = pd.melt(data.reset_index(),id_vars='date',var_name='attribute',value_name='value')
melted

,date,attribute,value
0,1959-03-31 23:59:59.999999999,realgdp,2710.349
1,1959-06-30 23:59:59.999999999,realgdp,2778.801
2,1959-09-30 23:59:59.999999999,realgdp,2775.488
3,1959-12-31 23:59:59.999999999,realgdp,2785.204
4,1960-03-31 23:59:59.999999999,realgdp,2847.699
...,...,...,...
604,2008-09-30 23:59:59.999999999,unemp,6.000
605,2008-12-31 23:59:59.999999999,unemp,6.900
606,2009-03-31 23:59:59.999999999,unemp,8.100
607,2009-06-30 23:59:59.999999999,unemp,9.200


In [ ]:
#above data frame has index ordered by attributes, suppose we want oreder by dates
#
melted = melted.sort_values('date').reset_index(drop=True)
melted

,date,attribute,value
0,1959-03-31 23:59:59.999999999,realgdp,2710.349
1,1959-03-31 23:59:59.999999999,infl,0.000
2,1959-03-31 23:59:59.999999999,unemp,5.800
3,1959-06-30 23:59:59.999999999,realgdp,2778.801
4,1959-06-30 23:59:59.999999999,infl,2.340
...,...,...,...
604,2009-06-30 23:59:59.999999999,infl,3.370
605,2009-06-30 23:59:59.999999999,realgdp,12901.504
606,2009-09-30 23:59:59.999999999,realgdp,12990.341
607,2009-09-30 23:59:59.999999999,infl,3.560


In [ ]:
melted = melted.set_index('date')
melted

,attribute,value
date,,
1959-03-31 23:59:59.999999999,realgdp,2710.349
1959-03-31 23:59:59.999999999,infl,0.000
1959-03-31 23:59:59.999999999,unemp,5.800
1959-06-30 23:59:59.999999999,realgdp,2778.801
1959-06-30 23:59:59.999999999,infl,2.340
...,...,...
2009-06-30 23:59:59.999999999,infl,3.370
2009-06-30 23:59:59.999999999,realgdp,12901.504
2009-09-30 23:59:59.999999999,realgdp,12990.341


In [ ]:
pd.pivot_table?

In [ ]:
# Difference between pivot and pivot_table, there are mainly two differences:

# pivot_table is a generalization of pivot that can handle duplicate values for one pivoted 
# index/column pair. Specifically, you can give pivot_table a list of aggregation functions 
# using keyword argument aggfunc. The default aggfunc of pivot_table is numpy.mean.
# pivot_table also supports using multiple columns for the index and column of the pivoted table. 
# A hierarchical index will be automatically generated for you.

# pivoted = pd.pivot(melted,'date','attribute','value')
# pivoted = pd.pivot(melted,index='date', columns='attribute', values='value')
# pivoted = pd.pivot_table(melted.reset_index(),index='date', columns='attribute', values='value')

# Now move back to the wide format
#
pivoted = pd.pivot_table(melted,index='date', columns='attribute', values='value')

pivoted

attribute,infl,realgdp,unemp
date,,,
1959-03-31 23:59:59.999999999,0.00,2710.349,5.8
1959-06-30 23:59:59.999999999,2.34,2778.801,5.1
1959-09-30 23:59:59.999999999,2.74,2775.488,5.3
1959-12-31 23:59:59.999999999,0.27,2785.204,5.6
1960-03-31 23:59:59.999999999,2.31,2847.699,5.2
...,...,...,...
2008-09-30 23:59:59.999999999,-3.16,13324.600,6.0
2008-12-31 23:59:59.999999999,-8.79,13141.920,6.9
2009-03-31 23:59:59.999999999,0.94,12925.410,8.1


In [ ]:
# Though a bit more difficult, we can achieve similar layouts with stacking. 
# here is the (wide) data
data

item,realgdp,infl,unemp
date,,,
1959-03-31 23:59:59.999999999,2710.349,0.00,5.8
1959-06-30 23:59:59.999999999,2778.801,2.34,5.1
1959-09-30 23:59:59.999999999,2775.488,2.74,5.3
1959-12-31 23:59:59.999999999,2785.204,0.27,5.6
1960-03-31 23:59:59.999999999,2847.699,2.31,5.2
...,...,...,...
2008-09-30 23:59:59.999999999,13324.600,-3.16,6.0
2008-12-31 23:59:59.999999999,13141.920,-8.79,6.9
2009-03-31 23:59:59.999999999,12925.410,0.94,8.1


In [ ]:
#Stack moves us from wide to narrow (EAV) with hierarchical index
ldata = data.stack() 
ldata

date                           item   
1959-03-31 23:59:59.999999999  realgdp     2710.349
                               infl           0.000
                               unemp          5.800
1959-06-30 23:59:59.999999999  realgdp     2778.801
                               infl           2.340
                                            ...    
2009-06-30 23:59:59.999999999  infl           3.370
                               unemp          9.200
2009-09-30 23:59:59.999999999  realgdp    12990.341
                               infl           3.560
                               unemp          9.600
Length: 609, dtype: float64

In [ ]:
# and back again into wide format
ldata.unstack()

item,realgdp,infl,unemp
date,,,
1959-03-31 23:59:59.999999999,2710.349,0.00,5.8
1959-06-30 23:59:59.999999999,2778.801,2.34,5.1
1959-09-30 23:59:59.999999999,2775.488,2.74,5.3
1959-12-31 23:59:59.999999999,2785.204,0.27,5.6
1960-03-31 23:59:59.999999999,2847.699,2.31,5.2
...,...,...,...
2008-09-30 23:59:59.999999999,13324.600,-3.16,6.0
2008-12-31 23:59:59.999999999,13141.920,-8.79,6.9
2009-03-31 23:59:59.999999999,12925.410,0.94,8.1


In [ ]:
# Reset the index of the DataFrame, and use the default one instead. 
# If the DataFrame has a MultiIndex, this method can remove one or more levels.
#
# EAV format now
# note how the date is replicated for each item
#
ldata = ldata.reset_index().rename(columns={0: 'value'})
ldata

,date,item,value
0,1959-03-31 23:59:59.999999999,realgdp,2710.349
1,1959-03-31 23:59:59.999999999,infl,0.000
2,1959-03-31 23:59:59.999999999,unemp,5.800
3,1959-06-30 23:59:59.999999999,realgdp,2778.801
4,1959-06-30 23:59:59.999999999,infl,2.340
...,...,...,...
604,2009-06-30 23:59:59.999999999,infl,3.370
605,2009-06-30 23:59:59.999999999,unemp,9.200
606,2009-09-30 23:59:59.999999999,realgdp,12990.341
607,2009-09-30 23:59:59.999999999,infl,3.560


In [ ]:
ldata = ldata.set_index('date')
ldata

,item,value
date,,
1959-03-31 23:59:59.999999999,realgdp,2710.349
1959-03-31 23:59:59.999999999,infl,0.000
1959-03-31 23:59:59.999999999,unemp,5.800
1959-06-30 23:59:59.999999999,realgdp,2778.801
1959-06-30 23:59:59.999999999,infl,2.340
...,...,...
2009-06-30 23:59:59.999999999,infl,3.370
2009-06-30 23:59:59.999999999,unemp,9.200
2009-09-30 23:59:59.999999999,realgdp,12990.341


In [ ]:

# following two fails: 
# pivoted = ldata.pivot('date', 'item', 'value')
# pivoted = ldata.pivot(index='date', columns='item', values='value')

# pivot_table command below moves us back to wide format 

pivoted = ldata.pivot_table(index='date', columns='item', values='value')
pivoted

item,infl,realgdp,unemp
date,,,
1959-03-31 23:59:59.999999999,0.00,2710.349,5.8
1959-06-30 23:59:59.999999999,2.34,2778.801,5.1
1959-09-30 23:59:59.999999999,2.74,2775.488,5.3
1959-12-31 23:59:59.999999999,0.27,2785.204,5.6
1960-03-31 23:59:59.999999999,2.31,2847.699,5.2
...,...,...,...
2008-09-30 23:59:59.999999999,-3.16,13324.600,6.0
2008-12-31 23:59:59.999999999,-8.79,13141.920,6.9
2009-03-31 23:59:59.999999999,0.94,12925.410,8.1


In [ ]:
pivoted.columns

Index(['infl', 'realgdp', 'unemp'], dtype='object', name='item')

In [ ]:
#will want to get the date back into a column for the melt function
pivoted.reset_index()

item,date,infl,realgdp,unemp
0,1959-03-31 23:59:59.999999999,0.00,2710.349,5.8
1,1959-06-30 23:59:59.999999999,2.34,2778.801,5.1
2,1959-09-30 23:59:59.999999999,2.74,2775.488,5.3
3,1959-12-31 23:59:59.999999999,0.27,2785.204,5.6
4,1960-03-31 23:59:59.999999999,2.31,2847.699,5.2
...,...,...,...,...
198,2008-09-30 23:59:59.999999999,-3.16,13324.600,6.0
199,2008-12-31 23:59:59.999999999,-8.79,13141.920,6.9
200,2009-03-31 23:59:59.999999999,0.94,12925.410,8.1
201,2009-06-30 23:59:59.999999999,3.37,12901.504,9.2


In [ ]:
#Pivot & melt work together to move back & forth Wide <--> narrow
#
pd.melt(pivoted.reset_index(),id_vars='date',var_name='attribute',value_name='value')

,date,attribute,value
0,1959-03-31 23:59:59.999999999,infl,0.00
1,1959-06-30 23:59:59.999999999,infl,2.34
2,1959-09-30 23:59:59.999999999,infl,2.74
3,1959-12-31 23:59:59.999999999,infl,0.27
4,1960-03-31 23:59:59.999999999,infl,2.31
...,...,...,...
604,2008-09-30 23:59:59.999999999,unemp,6.00
605,2008-12-31 23:59:59.999999999,unemp,6.90
606,2009-03-31 23:59:59.999999999,unemp,8.10
607,2009-06-30 23:59:59.999999999,unemp,9.20


In [ ]:
ldata['value2'] = np.random.randn(len(ldata))
ldata[:10]

,item,value,value2
date,,,
1959-03-31 23:59:59.999999999,realgdp,2710.349,-1.127830
1959-03-31 23:59:59.999999999,infl,0.000,1.773196
1959-03-31 23:59:59.999999999,unemp,5.800,1.340844
1959-06-30 23:59:59.999999999,realgdp,2778.801,1.036512
1959-06-30 23:59:59.999999999,infl,2.340,-0.811384
1959-06-30 23:59:59.999999999,unemp,5.100,-0.216927
1959-09-30 23:59:59.999999999,realgdp,2775.488,-0.652381
1959-09-30 23:59:59.999999999,infl,2.740,0.398269
1959-09-30 23:59:59.999999999,unemp,5.300,2.135853


In [ ]:
ldata.columns

Index(['item', 'value', 'value2'], dtype='object')

In [ ]:
# Not quite what we want
ldata.stack()

date                                 
1959-03-31 23:59:59.999999999  item       realgdp
                               value      2710.35
                               value2    -1.12783
                               item          infl
                               value            0
                                           ...   
2009-09-30 23:59:59.999999999  value         3.56
                               value2   -0.242161
                               item         unemp
                               value          9.6
                               value2     1.31438
Length: 1827, dtype: object

In [ ]:
# Move to the wide format .... but ! we have two columns for values EAVV
# again, plain old pivot does not work

pivoted = ldata.pivot_table(index='date', columns='item')
pivoted[:5]

value                    value2            \
item                           infl   realgdp unemp      infl   realgdp   
date                                                                      
1959-03-31 23:59:59.999999999  0.00  2710.349   5.8  1.773196 -1.127830   
1959-06-30 23:59:59.999999999  2.34  2778.801   5.1 -0.811384  1.036512   
1959-09-30 23:59:59.999999999  2.74  2775.488   5.3  0.398269 -0.652381   
1959-12-31 23:59:59.999999999  0.27  2785.204   5.6 -0.819642  2.714261   
1960-03-31 23:59:59.999999999  2.31  2847.699   5.2  0.398959 -0.952553   

                                         
item                              unemp  
date                                     
1959-03-31 23:59:59.999999999  1.340844  
1959-06-30 23:59:59.999999999 -0.216927  
1959-09-30 23:59:59.999999999  2.135853  
1959-12-31 23:59:59.999999999  0.627482  
1960-03-31 23:59:59.999999999  0.001305

In [ ]:
pivoted['value'][:5]

item,infl,realgdp,unemp
date,,,
1959-03-31 23:59:59.999999999,0.00,2710.349,5.8
1959-06-30 23:59:59.999999999,2.34,2778.801,5.1
1959-09-30 23:59:59.999999999,2.74,2775.488,5.3
1959-12-31 23:59:59.999999999,0.27,2785.204,5.6
1960-03-31 23:59:59.999999999,2.31,2847.699,5.2


In [ ]:
ldata.index

DatetimeIndex(['1959-03-31 23:59:59.999999999',
               '1959-03-31 23:59:59.999999999',
               '1959-03-31 23:59:59.999999999',
               '1959-06-30 23:59:59.999999999',
               '1959-06-30 23:59:59.999999999',
               '1959-06-30 23:59:59.999999999',
               '1959-09-30 23:59:59.999999999',
               '1959-09-30 23:59:59.999999999',
               '1959-09-30 23:59:59.999999999',
               '1959-12-31 23:59:59.999999999',
               ...
               '2008-12-31 23:59:59.999999999',
               '2009-03-31 23:59:59.999999999',
               '2009-03-31 23:59:59.999999999',
               '2009-03-31 23:59:59.999999999',
               '2009-06-30 23:59:59.999999999',
               '2009-06-30 23:59:59.999999999',
               '2009-06-30 23:59:59.999999999',
               '2009-09-30 23:59:59.999999999',
               '2009-09-30 23:59:59.999999999',
               '2009-09-30 23:59:59.999999999'],
              dtype=

In [ ]:
# labeled as unstacked as we have take items out of hierarchical columns

unstacked = ldata.set_index(['item'], append=True)
unstacked[:7]

value    value2
date                          item                       
1959-03-31 23:59:59.999999999 realgdp  2710.349 -1.127830
                              infl        0.000  1.773196
                              unemp       5.800  1.340844
1959-06-30 23:59:59.999999999 realgdp  2778.801  1.036512
                              infl        2.340 -0.811384
                              unemp       5.100 -0.216927
1959-09-30 23:59:59.999999999 realgdp  2775.488 -0.652381

In [ ]:
unstacked.index

MultiIndex([('1959-03-31 23:59:59.999999999', 'realgdp'),
            ('1959-03-31 23:59:59.999999999',    'infl'),
            ('1959-03-31 23:59:59.999999999',   'unemp'),
            ('1959-06-30 23:59:59.999999999', 'realgdp'),
            ('1959-06-30 23:59:59.999999999',    'infl'),
            ('1959-06-30 23:59:59.999999999',   'unemp'),
            ('1959-09-30 23:59:59.999999999', 'realgdp'),
            ('1959-09-30 23:59:59.999999999',    'infl'),
            ('1959-09-30 23:59:59.999999999',   'unemp'),
            ('1959-12-31 23:59:59.999999999', 'realgdp'),
            ...
            ('2008-12-31 23:59:59.999999999',   'unemp'),
            ('2009-03-31 23:59:59.999999999', 'realgdp'),
            ('2009-03-31 23:59:59.999999999',    'infl'),
            ('2009-03-31 23:59:59.999999999',   'unemp'),
            ('2009-06-30 23:59:59.999999999', 'realgdp'),
            ('2009-06-30 23:59:59.999999999',    'infl'),
            ('2009-06-30 23:59:59.999999999',   'unemp')

In [ ]:
unstacked.loc[('1959-03-31 23:59:59.999999999', 'realgdp')]

value     2710.34900
value2      -1.12783
Name: (1959-03-31 23:59:59.999999999, realgdp), dtype: float64

In [ ]:
unstacked.loc[('1959-03-31 23:59:59.999999999', 'realgdp'),"value2"]

date                           item   
1959-03-31 23:59:59.999999999  realgdp   -1.12783
Name: value2, dtype: float64

In [ ]:
unstacked.loc['1959-03-31 23:59:59.99999999']

value    value2
date                          item                       
1959-03-31 23:59:59.999999999 realgdp  2710.349 -1.127830
                              infl        0.000  1.773196
                              unemp       5.800  1.340844

In [ ]:
unstacked["value2"]

date                           item   
1959-03-31 23:59:59.999999999  realgdp   -1.127830
                               infl       1.773196
                               unemp      1.340844
1959-06-30 23:59:59.999999999  realgdp    1.036512
                               infl      -0.811384
                                            ...   
2009-06-30 23:59:59.999999999  infl      -1.502196
                               unemp     -0.115303
2009-09-30 23:59:59.999999999  realgdp   -1.638873
                               infl      -0.242161
                               unemp      1.314378
Name: value2, Length: 609, dtype: float64

In [ ]:
unstacked.iloc[1]

value     0.000000
value2    1.773196
Name: (1959-03-31 23:59:59.999999999, infl), dtype: float64

In [ ]:
# Suppose we stacked the values ?
stack2 = unstacked.stack()
stack2

date                           item           
1959-03-31 23:59:59.999999999  realgdp  value     2710.349000
                                        value2      -1.127830
                               infl     value        0.000000
                                        value2       1.773196
                               unemp    value        5.800000
                                                     ...     
2009-09-30 23:59:59.999999999  realgdp  value2      -1.638873
                               infl     value        3.560000
                                        value2      -0.242161
                               unemp    value        9.600000
                                        value2       1.314378
Length: 1218, dtype: float64

In [ ]:
stack2.index

MultiIndex([('1959-03-31 23:59:59.999999999', 'realgdp',  'value'),
            ('1959-03-31 23:59:59.999999999', 'realgdp', 'value2'),
            ('1959-03-31 23:59:59.999999999',    'infl',  'value'),
            ('1959-03-31 23:59:59.999999999',    'infl', 'value2'),
            ('1959-03-31 23:59:59.999999999',   'unemp',  'value'),
            ('1959-03-31 23:59:59.999999999',   'unemp', 'value2'),
            ('1959-06-30 23:59:59.999999999', 'realgdp',  'value'),
            ('1959-06-30 23:59:59.999999999', 'realgdp', 'value2'),
            ('1959-06-30 23:59:59.999999999',    'infl',  'value'),
            ('1959-06-30 23:59:59.999999999',    'infl', 'value2'),
            ...
            ('2009-06-30 23:59:59.999999999',    'infl',  'value'),
            ('2009-06-30 23:59:59.999999999',    'infl', 'value2'),
            ('2009-06-30 23:59:59.999999999',   'unemp',  'value'),
            ('2009-06-30 23:59:59.999999999',   'unemp', 'value2'),
            ('2009-09-30 23:59:5